In [2]:
import os, sys
import pandas as pd
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import datetime
from datetime import date
%matplotlib inline

In [7]:
data_dir = '../Data/'
data_filedir = data_dir + '2016_Jan.csv'

In [8]:
data = pd.read_table(data_filedir, sep='\t', error_bad_lines=False)
data.head()

b'Skipping line 596959: expected 14 fields, saw 26\n'


,dock_id,dock_name,date,hour,minute,pm,avail_bikes,avail_docks,tot_docks,_lat,_long,in_service,status_key,dock_name6
0,72,W 52 St & 11 Ave,16-01-01,1,12,0,10,27,39,40.767272,-73.993929,1,1,NaN
1,72,W 52 St & 11 Ave,16-01-01,1,24,0,10,27,39,40.767272,-73.993929,1,1,NaN
2,72,W 52 St & 11 Ave,16-01-01,1,35,0,10,27,39,40.767272,-73.993929,1,1,NaN
3,72,W 52 St & 11 Ave,16-01-01,1,46,0,10,27,39,40.767272,-73.993929,1,1,NaN
4,72,W 52 St & 11 Ave,16-01-01,1,57,0,9,28,39,40.767272,-73.993929,1,1,NaN


In [9]:
data.drop('dock_name6', axis = 1, inplace = True)

In [10]:
data['datetime'] = pd.to_datetime(data['date'], format='%y-%m-%d')
# data.head()
data.drop('date', axis = 1, inplace = True)

In [11]:
data['minute_bin'] = pd.cut(data['minute'], bins=[0, 16, 31, 46, 59], include_lowest=True, labels=['1', '2', '3', '4'])
data.head()

,dock_id,dock_name,hour,minute,pm,avail_bikes,avail_docks,tot_docks,_lat,_long,in_service,status_key,datetime,minute_bin
0,72,W 52 St & 11 Ave,1,12,0,10,27,39,40.767272,-73.993929,1,1,2016-01-01,1
1,72,W 52 St & 11 Ave,1,24,0,10,27,39,40.767272,-73.993929,1,1,2016-01-01,2
2,72,W 52 St & 11 Ave,1,35,0,10,27,39,40.767272,-73.993929,1,1,2016-01-01,3
3,72,W 52 St & 11 Ave,1,46,0,10,27,39,40.767272,-73.993929,1,1,2016-01-01,3
4,72,W 52 St & 11 Ave,1,57,0,9,28,39,40.767272,-73.993929,1,1,2016-01-01,4


In [12]:
key = [data.dock_id, data.datetime, data.pm, data.hour, data.minute_bin, data._lat, data._long, data.in_service, data.status_key]
data = data['avail_bikes'].groupby(key).min().reset_index()

In [15]:
# keep rush hours 6-10am and 4-8pm
data_morning = data[(data['pm'] == 0) & (data['hour'].isin(range(6,11)))]
# data_morning.head()

In [16]:
data_afternoon = data[(data['pm'] == 1) & (data['hour'].isin(range(4,9)))]
# data_afternoon.head()

In [17]:
data = pd.concat([data_morning, data_afternoon])

In [18]:
data['shortage'] = [1 if x<=5 else 0 for x in data['avail_bikes']]

In [20]:
data['day_of_week'] = data['datetime'].dt.weekday_name
data.head()

,dock_id,datetime,pm,hour,minute_bin,_lat,_long,in_service,status_key,avail_bikes,shortage,day_of_week
20,72,2016-01-01,0,6,1,40.767272,-73.993929,1,1,8,0,Friday
21,72,2016-01-01,0,6,2,40.767272,-73.993929,1,1,8,0,Friday
22,72,2016-01-01,0,6,3,40.767272,-73.993929,1,1,8,0,Friday
23,72,2016-01-01,0,6,4,40.767272,-73.993929,1,1,8,0,Friday
24,72,2016-01-01,0,7,1,40.767272,-73.993929,1,1,8,0,Friday


In [21]:
# days = {'weekday':['Monday', 'Tuesday', 'Wednesday', 'Thursday'],'weekend':['Friday', 'Saturday', 'Sunday']}

data['weekday'] = [0 if x in ['Saturday','Sunday'] else 1 for x in data['day_of_week']]

In [22]:
holiday = ['2016-01-01','2016-01-18', '2016-02-15', '2016-05-30',
           '2016-07-04', '2016-09-05', '2016-10-10', '2016-11-11', '2016-11-24', '2016-12-25']

In [23]:
holiday = pd.to_datetime(holiday, format='%Y-%m-%d')
holiday

DatetimeIndex(['2016-01-01', '2016-01-18', '2016-02-15', '2016-05-30',
               '2016-07-04', '2016-09-05', '2016-10-10', '2016-11-11',
               '2016-11-24', '2016-12-25'],
              dtype='datetime64[ns]', freq=None)

In [24]:
data['holiday'] = [1 if x in holiday else 0 for x in data['datetime']]

In [25]:
data.head()

,dock_id,datetime,pm,hour,minute_bin,_lat,_long,in_service,status_key,avail_bikes,shortage,day_of_week,weekday,holiday
20,72,2016-01-01,0,6,1,40.767272,-73.993929,1,1,8,0,Friday,1,1
21,72,2016-01-01,0,6,2,40.767272,-73.993929,1,1,8,0,Friday,1,1
22,72,2016-01-01,0,6,3,40.767272,-73.993929,1,1,8,0,Friday,1,1
23,72,2016-01-01,0,6,4,40.767272,-73.993929,1,1,8,0,Friday,1,1
24,72,2016-01-01,0,7,1,40.767272,-73.993929,1,1,8,0,Friday,1,1
